In [1]:

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer


# In[2]:


filmes = pd.read_csv("movies_metadata.csv")
creditos = pd.read_csv("credits.csv")

#Usando as palavras chaves no dataset keywords

palavraschaves = pd.read_csv("keywords.csv")


# In[3]:


filmes.head()


# In[4]:


filmes = filmes[["id", "title", "genres"]]


# In[5]:


#Função de limpar os ID's

def limpa_id(x):
    
    
    try: 
        
        return int(x)
    
    except:
        
        
        return np.nan
    
    


# In[6]:


filmes.head()


# In[7]:


filmes['id'] = filmes['id'].apply(limpa_id)
filmes = filmes[filmes['id'].notnull()]


# In[8]:


filmes['id'] = filmes['id'].astype('int')
palavraschaves['id'] = palavraschaves['id']
creditos['id'] = creditos['id'].astype('int')


# In[9]:


#Juntando através do id

filmes1 = filmes.merge(creditos, on = "id")
filmes2 = filmes1.merge(palavraschaves, on = "id")


# In[10]:


filmes.head()


# In[11]:


filmes1.head()


# In[12]:


filmes2.head()


# In[13]:


creditos.head()


# In[14]:


# aplicando literal_eval para fazer string -> objetos.

filmes2['genres'] = filmes2['genres'].apply(literal_eval)


# In[15]:


filmes2.head()


# In[17]:


filmes2.info()


# In[18]:


filmes2['cast'] = filmes2['cast'].apply(literal_eval)


# In[19]:


filmes2['crew'] = filmes2['crew'].apply(literal_eval)
filmes2['keywords'] = filmes2['keywords'].apply(literal_eval)


# In[20]:


#pegando o nome dos generos associados
filmes2['crew'] = filmes2['crew'].apply(lambda x: [i['name'].lower() for i in x if i['job'] == 'Director'])


# In[25]:


#juntando com as palavras chaves
df = filmes2


# In[26]:


df.head()


C:\Users\lucas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46628 non-null  int32 
 1   title     46624 non-null  object
 2   genres    46628 non-null  object
 3   cast      46628 non-null  object
 4   crew      46628 non-null  object
 5   keywords  46628 non-null  object
dtypes: int32(1), object(5)
memory usage: 2.3+ MB


,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...",[john lasseter],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'cast_id': 1, 'character': 'Alan Parrish', '...",[joe johnston],"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[howard deutch],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'cast_id': 1, 'character': 'Savannah 'Vannah...",[forest whitaker],"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'cast_id': 1, 'character': 'George Banks', '...",[charles shyer],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [2]:
df["genres"] = df["genres"].apply(lambda x: [i["name"].lower() for i in x])

In [3]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[{'cast_id': 14, 'character': 'Woody (voice)',...",[john lasseter],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[adventure, fantasy, family]","[{'cast_id': 1, 'character': 'Alan Parrish', '...",[joe johnston],"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[romance, comedy]","[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[howard deutch],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[{'cast_id': 1, 'character': 'Savannah 'Vannah...",[forest whitaker],"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,[comedy],"[{'cast_id': 1, 'character': 'George Banks', '...",[charles shyer],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [4]:
df["cast"] = df["cast"].apply(lambda x: [i["name"].lower() for i in x])


In [6]:
df["keywords"] = df["keywords"].apply(lambda x: [i["name"].lower() for i in x])


In [7]:
#pegando até 3 caracteristicas de cada filme
# se pegarmos muitas caracteristicas pode aumentar demais a complexidade do algortimo

df["genre"] = df["genres"].apply(lambda x: x[:3] if len(x)>3 else x)
df["cast"] = df["cast"].apply(lambda x: x[:3] if len(x)>3 else x)
df["keywords"] = df["keywords"].apply(lambda x: x[:3] if len(x)>3 else x)

In [9]:
df.head()

,id,title,genres,cast,crew,keywords,genre
0,862,Toy Story,"[animation, comedy, family]","[tom hanks, tim allen, don rickles]",[john lasseter],"[jealousy, toy, boy]","[animation, comedy, family]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robin williams, jonathan hyde, kirsten dunst]",[joe johnston],"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,15602,Grumpier Old Men,"[romance, comedy]","[walter matthau, jack lemmon, ann-margret]",[howard deutch],"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitney houston, angela bassett, loretta devine]",[forest whitaker],"[based on novel, interracial relationship, sin...","[comedy, drama, romance]"
4,11862,Father of the Bride Part II,[comedy],"[steve martin, diane keaton, martin short]",[charles shyer],"[baby, midlife crisis, confidence]",[comedy]


In [11]:
df = df[['id', 'title', 'genres', 'cast', 'crew', 'keywords']]

In [12]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tom hanks, tim allen, don rickles]",[john lasseter],"[jealousy, toy, boy]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robin williams, jonathan hyde, kirsten dunst]",[joe johnston],"[board game, disappearance, based on children'..."
2,15602,Grumpier Old Men,"[romance, comedy]","[walter matthau, jack lemmon, ann-margret]",[howard deutch],"[fishing, best friend, duringcreditsstinger]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitney houston, angela bassett, loretta devine]",[forest whitaker],"[based on novel, interracial relationship, sin..."
4,11862,Father of the Bride Part II,[comedy],"[steve martin, diane keaton, martin short]",[charles shyer],"[baby, midlife crisis, confidence]"


In [16]:
#removendo os espaços

df["cast"] = df["cast"].apply(lambda x: [i.replace(" ","") for i in x])
df["crew"] = df["crew"].apply(lambda x: [i.replace(" ","") for i in x])
df["keywords"] = df["keywords"].apply(lambda x: [i.replace(" ","") for i in x])
df["genres"] = df["genres"].apply(lambda x: [i.replace(" ","") for i in x])

<ipython-input-16-699c6c87148b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cast"] = df["cast"].apply(lambda x: [i.replace(" ","") for i in x])
<ipython-input-16-699c6c87148b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crew"] = df["crew"].apply(lambda x: [i.replace(" ","") for i in x])
<ipython-input-16-699c6c87148b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [17]:
df.head()

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]"
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single..."
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]"


In [19]:
#Transformando tudo em uma coluna só 

df["metadata"] = df.apply(lambda x : " ".join(x["genres"]) + " "  + " ".join(x["cast"]) + " " + " ".join(x["crew"]) + " " + " ".join(x["keywords"]), axis = 1)

In [20]:
df.head()

,id,title,genres,cast,crew,keywords,metadata
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy]",animation comedy family tomhanks timallen donr...
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbook]",adventure fantasy family robinwilliams jonatha...
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger]",romance comedy waltermatthau jacklemmon ann-ma...
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single...",comedy drama romance whitneyhouston angelabass...
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence]",comedy stevemartin dianekeaton martinshort cha...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46628 non-null  int32 
 1   title     46624 non-null  object
 2   genres    46628 non-null  object
 3   cast      46628 non-null  object
 4   crew      46628 non-null  object
 5   keywords  46628 non-null  object
 6   metadata  46628 non-null  object
dtypes: int32(1), object(6)
memory usage: 2.7+ MB


In [39]:
df_metadata = df.iloc[1:4000, 6]

In [40]:
df_metadata

1       adventure fantasy family robinwilliams jonatha...
2       romance comedy waltermatthau jacklemmon ann-ma...
3       comedy drama romance whitneyhouston angelabass...
4       comedy stevemartin dianekeaton martinshort cha...
5       action crime drama thriller alpacino robertden...
                              ...                        
3995    comedy richardpryor rachelticotin rubénblades ...
3996    drama denzelwashington kevinkline penelopewilt...
3997    horror susanbrandy mariaisabellopez mariayuenc...
3998    horror thriller crime mystery marysteenburgen ...
3999    drama anjelicahuston donalmccann dano'herlihy ...
Name: metadata, Length: 3999, dtype: object

In [41]:
#Usando cosseno de similaridade para achar a semelhança entre dois filmes

cv = CountVectorizer(stop_words = 'english')

contador_matrix = cv.fit_transform(df_metadata)

cosseno_sim_matrix = cosine_similarity(contador_matrix)

In [42]:
cosseno_sim_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.2       , 1.        , ..., 0.        , 0.        ,
        0.11952286],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.13608276,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.13608276, 1.        ,
        0.        ],
       [0.        , 0.        , 0.11952286, ..., 0.        , 0.        ,
        1.        ]])

In [44]:
len(cosseno_sim_matrix)

3999

In [47]:
#Mapeando a função 

mapear = pd.Series(df_metadata.index, index = df.iloc[1:4000, 1])
mapear

title
Jumanji                           1
Grumpier Old Men                  2
Waiting to Exhale                 3
Father of the Bride Part II       4
Heat                              5
                               ... 
Critical Condition             3995
Cry Freedom                    3996
Curse                          3997
Dead of Winter                 3998
The Dead                       3999
Length: 3999, dtype: int64

In [48]:
df_metadata

1       adventure fantasy family robinwilliams jonatha...
2       romance comedy waltermatthau jacklemmon ann-ma...
3       comedy drama romance whitneyhouston angelabass...
4       comedy stevemartin dianekeaton martinshort cha...
5       action crime drama thriller alpacino robertden...
                              ...                        
3995    comedy richardpryor rachelticotin rubénblades ...
3996    drama denzelwashington kevinkline penelopewilt...
3997    horror susanbrandy mariaisabellopez mariayuenc...
3998    horror thriller crime mystery marysteenburgen ...
3999    drama anjelicahuston donalmccann dano'herlihy ...
Name: metadata, Length: 3999, dtype: object

In [57]:
# Funcao de recomendacao de acordo com nossos metadados

def sistema(filmes_input):
    
    filme_indexado = mapear[filmes_input]
    
    #obtendo os valores similares
    
    score = list(enumerate (cosseno_sim_matrix[filme_indexado]))
    
    score = sorted(score, key = lambda x: x[1], reverse = True)
    
    # Amostra de 20 filmes
    
    
    score = score[1:20]
    
    indices = [i[0] for i in score]
    
    return (mapear.iloc[indices])


In [58]:
sistema("Frisk")

title
Houseguest                         244
Jingle All the Way                1332
Parenthood                        3432
Leave it to Beaver                1550
Ri¢hie Ri¢h                        369
Coneheads                          430
Nothing but Trouble               2173
My Favorite Martian               2405
Antz                              2201
The Brave Little Toaster          3991
A Simple Twist of Fate             531
Larger Than Life                   798
The Apple Dumpling Gang            999
The Million Dollar Duck           1942
The Barefoot Executive            1943
Blank Check                       1947
The Computer Wore Tennis Shoes    1951
Superstar                         2818
Snow Day                          3195
dtype: int64